# Construção da ABT
_ABT - Analytical Base Table_

---

## Sumário

1. **Importação da bibliotecas**
2. **Criação e iniciação de uma sessão PySpark**
3. **Criação dos datasets a partir da leitura dos arquivos *.csv e *.json**

<br>

---

<br>

## 1. Importação de bibliotecas

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, expr
import json
import csv

## 2. Criação e iniciação de uma sessão PySpark

In [11]:
appName = 'ABT - Transações de clientes'

# O objeto SparkSession configurado com configurações específicas
spark = SparkSession.builder \
    .appName(appName) \
    .config('spark.driver.memory', '8g') \
    .config('spark.executor.memory', '8g') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()
    
spark.sparkContext.setLogLevel('ERROR')

spark

## 3. Criação dos datasets a partir da leitura dos arquivos *.csv e *.json

In [12]:
# Caminho dos arquivos *.csv e *.json
caminho = 'dados/'

In [13]:
# Dados de cartão de crédito
df_cards = spark.read.csv(caminho + 'cards_data.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_cards.printSchema()

root
 |-- id: integer (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_brand: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- card_number: long (nullable = true)
 |-- expires: string (nullable = true)
 |-- cvv: integer (nullable = true)
 |-- has_chip: string (nullable = true)
 |-- num_cards_issued: integer (nullable = true)
 |-- credit_limit: string (nullable = true)
 |-- acct_open_date: string (nullable = true)
 |-- year_pin_last_changed: integer (nullable = true)
 |-- card_on_dark_web: string (nullable = true)



In [14]:
# Dados de transações
df_transactions = spark.read.csv(caminho + 'transactions_data.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_transactions.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_id: integer (nullable = true)
 |-- amount: string (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_id: integer (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: double (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- errors: string (nullable = true)



In [15]:
# Dados de clientes
df_clients = spark.read.csv(caminho + 'users_data.csv', header=True, inferSchema=True)

# Exibir o esquema do dataframe
df_clients.printSchema()

root
 |-- id: integer (nullable = true)
 |-- current_age: integer (nullable = true)
 |-- retirement_age: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- birth_month: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- per_capita_income: string (nullable = true)
 |-- yearly_income: string (nullable = true)
 |-- total_debt: string (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- num_credit_cards: integer (nullable = true)



In [32]:
def process_json_to_csv(json_path, csv_path, list_columns, one_column=None):
    '''
    Processa um arquivo JSON e converte em CSV.

    :param json_path: path + filename.json
        Caminho do arquivo JSON de entrada.
    :param csv_path: path + filename.csv
        Caminho do arquivo CSV de saída.
    :param list_columns: list
        Lista com os nomes das colunas do CSV.
    :param one_column: string (default=None)
        Nome da chave no JSON para acessar dados aninhados (opcional).
    '''
    # Carregar o arquivo JSON
    with open(json_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        
    if not data:
        print('Arquivo JSON está vazio.')
        return
    
    # Processar dados aninhados, se necessário
    if one_column:
        data = data.get(one_column, {})
        if not data:
            print(f'A chave \'{one_column}\' não foi encontrada ou está vazia no JSON.')
            return
        print(f'Dados extraídos da chave \'{one_column}\'.')
    else:
        print('Processando JSON sem dados aninhados.')

    # Escrever o CSV
    with open(csv_path, 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(list_columns)  # Escrevendo o cabeçalho
        
        for key, value in data.items():
            writer.writerow([key, value])  # Escrevendo as linhas
    
    print(f'Arquivo CSV gerado com sucesso em {csv_path}.')

In [33]:
# Processar mcc_codes.json
json_path = caminho + 'mcc_codes.json'
csv_path = caminho + 'mcc_codes.csv'
list_columns = ['code', 'description']
process_json_to_csv(json_path, csv_path, list_columns)

Processando JSON sem dados aninhados.
Arquivo CSV gerado com sucesso em dados/mcc_codes.csv.


In [34]:
# Processar train_fraud_labels.json
json_path = caminho + 'train_fraud_labels.json'
csv_path = caminho + 'train_fraud_labels.csv'
list_columns = ['transaction_id', 'is_fraud']
one_column = 'target'
process_json_to_csv(json_path, csv_path, list_columns, one_column)

Dados extraídos da chave 'target'.
Arquivo CSV gerado com sucesso em dados/train_fraud_labels.csv.


In [ ]:
import sys
sys.exit()